In [5]:
from sqlalchemy import create_engine
import pymysql

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/kaggle_hour_price', pool_recycle=3600)
dbConnection    = sqlEngine.connect()

In [6]:
%run package_import.ipynb

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn import preprocessing
from sklearn.metrics import classification_report, mean_absolute_error

In [8]:
df_combined = pd.read_sql_query(

''' 
select *
from transformed_combined
'''
,dbConnection)

In [9]:
df_profile = pd.read_sql_query(

''' 
select *
from raw_data_profile
'''
,dbConnection)

df_profile

,column_names,col_desc,value_desc,data_type,unique_count,min_value,max_value,mean_value,median_value,null_values,total_rows,%_null_values,num_lower_outliers,num_higher_outliers
0,Id,None,nan,int64,2919,1.0,2919.0,1460.000000,1460.0,0,2919,0.000000,0.0,0.0
1,MSSubClass,Identifies the type of dwelling involved in the sale.,"{'20': '1-STORY 1946 & NEWER ALL STYLES', '30': '1-STORY 1945 & OLDER', '40': '1-STORY W/FINISHED ATTIC ALL AGES', '45': '1-1/2 STORY - UNFINISHED ALL AGES', '50': '1-1/2 STORY FINISHED ALL AGES', '60': '2-STORY 1946 & NEWER', '70': '2-STORY 1945 & OLDER', '75': '2-1/2 STORY ALL AGES', '80': 'SPLIT OR MULTI-LEVEL', '85': 'SPLIT FOYER', '90': 'DUPLEX - ALL STYLES AND AGES', '120': '1-STORY PUD (Planned Unit Development) - 1946 & NEWER', '150': '1-1/2 STORY PUD - ALL AGES', '160': '2-STORY PUD - 1946 & NEWER', '180': 'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER', '190': '2 FAMILY CONVERSION - ALL STYLES AND AGES'}",int64,16,20.0,190.0,57.137718,50.0,0,2919,0.000000,0.0,207.0
2,MSZoning,Identifies the general zoning classification of the sale.,"{'A': 'Agriculture', 'C': 'Commercial', 'FV': 'Floating Village Residential', 'I': 'Industrial', 'RH': 'Residential High Density', 'RL': 'Residential Low Density', 'RP': 'Residential Low Density Park', 'RM': 'Residential Medium Density'}",object,6,NaN,NaN,NaN,NaN,4,2919,0.001370,NaN,NaN
3,LotFrontage,Linear feet of street connected to property,{},float64,129,21.0,313.0,69.305795,68.0,486,2919,0.166495,104.0,94.0
4,LotArea,Lot size in square feet,{},int64,1951,1300.0,215245.0,10168.114080,9453.0,0,2919,0.000000,1.0,127.0
5,Street,Type of road access to property,"{'Grvl': 'Gravel', 'Pave': 'Paved'}",object,2,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
6,Alley,Type of alley access to property,"{'Grvl': 'Gravel', 'Pave': 'Paved', 'NA': 'No alley access'}",object,3,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
7,LotShape,General shape of property,"{'Reg': 'Regular', 'IR1': 'Slightly irregular', 'IR2': 'Moderately Irregular', 'IR3': 'Irregular'}",object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
8,LandContour,Flatness of the property,"{'Lvl': 'Near Flat/Level', 'Bnk': 'Banked - Quick and significant rise from street grade to building', 'HLS': 'Hillside - Significant slope from side to side', 'Low': 'Depression'}",object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
9,Utilities,Type of utilities available,"{'AllPub': 'All public Utilities (E,G,W,& S)', 'NoSewr': 'Electricity, Gas, and Water (Septic Tank)', 'NoSeWa': 'Electricity and Gas Only', 'ELO': 'Electricity only'}",object,3,NaN,NaN,NaN,NaN,2,2919,0.000685,NaN,NaN


### Read train data

In [10]:
df_train = pd.read_sql_query(

''' 
select *
from transformed_combined
where source = 'train'
'''
,dbConnection)

df_train.drop(columns=['source'], inplace=True)


In [11]:
df_train['Utilities'].unique()

array(['AllPub', 'NoSeWa'], dtype=object)

### Prepare data for ML

In [55]:
cols_selected = ['Id',
        'LotArea', 'LandContour', 'Utilities',
        'LandSlope', 'BldgType', 'OverallQual',
        'OverallCond', 'ExterQual', 'ExterCond', 
        'MoSold','SalePrice'        
                ]
df_train = df_train[cols_selected]
X = df_train.iloc[:, :-1]
y = df_train.iloc[:, -1]

X = pd.get_dummies(X)
X.drop(columns=['Id', 'LandSlope_Gtl', 'ExterQual_TA'], inplace=True)
# X = np.array(X)
# y = np.array(y)

In [54]:
X.corr()

,LotArea,OverallQual,OverallCond,MoSold,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA
LotArea,1.000000,0.105806,-0.005636,0.001205,0.028095,0.089605,0.359475,-0.256843,-0.010123,0.010123,-0.310711,0.092649,0.540380,0.163554,0.077767,-0.015346,-0.141863,-0.167717,0.076372,-0.021622,0.003151,-0.027890,0.001447,-0.028664,0.001327,-0.013557,0.011682
OverallQual,0.105806,1.000000,-0.091932,0.070815,-0.083523,0.103611,-0.046563,0.017672,0.001881,-0.001881,0.063330,-0.049135,-0.043725,0.035872,-0.123989,-0.158156,-0.003723,0.128662,0.384495,-0.164663,0.547731,-0.646247,0.040502,-0.150923,-0.075139,-0.058690,0.131245
OverallCond,-0.005636,-0.091932,1.000000,-0.003511,0.062886,-0.002597,-0.034591,-0.022931,-0.009994,0.009994,-0.016765,0.022696,-0.009697,0.154637,0.022057,-0.112683,-0.017261,-0.136749,-0.022983,-0.114071,-0.174598,0.201418,0.139695,-0.139661,0.334538,-0.084144,-0.260800
MoSold,0.001205,0.070815,-0.003511,1.000000,-0.036519,0.070935,0.030087,-0.033515,0.051552,-0.051552,-0.004384,0.000093,0.010292,0.028213,-0.054460,0.037276,-0.007259,-0.030886,0.003091,-0.027323,0.044001,-0.038468,0.022577,-0.046220,0.001689,-0.012805,0.015724
LandContour_Bnk,0.028095,-0.083523,0.062886,-0.036519,1.000000,-0.039990,-0.033765,-0.629913,0.005560,-0.005560,-0.189379,0.199310,0.015754,0.003239,0.109016,0.031941,-0.036993,-0.061802,-0.004435,0.152049,-0.129033,0.096594,-0.009636,0.019460,0.007865,-0.005560,-0.013590
LandContour_HLS,0.089605,0.103611,-0.002597,0.070935,-0.039990,1.000000,-0.029941,-0.558578,0.004930,-0.004930,-0.256723,0.251521,0.062334,0.002227,-0.001610,0.004454,-0.032804,0.015383,0.065418,0.020117,0.018265,-0.046781,-0.008545,0.001129,-0.012554,-0.004930,0.012615
LandContour_Low,0.359475,-0.046563,-0.034591,0.030087,-0.033765,-0.029941,1.000000,-0.471633,0.004163,-0.004163,-0.492511,0.351135,0.408068,0.034771,0.007219,0.017106,-0.027698,-0.046273,-0.030556,0.120318,-0.000308,-0.012194,-0.007215,0.042173,-0.008833,-0.004163,-0.008247
LandContour_Lvl,-0.256843,0.017672,-0.022931,-0.033515,-0.629913,-0.558578,-0.471633,1.000000,-0.008826,0.008826,0.533652,-0.464768,-0.257070,-0.021325,-0.075906,-0.032879,0.058727,0.055947,-0.020671,-0.175783,0.075796,-0.030487,0.015298,-0.035344,0.006788,0.008826,0.005769
Utilities_AllPub,-0.010123,0.001881,-0.009994,0.051552,0.005560,0.004930,0.004163,-0.008826,1.000000,-1.000000,-0.006220,0.005651,0.002481,-0.011612,0.003856,0.005031,0.004561,0.007619,0.005031,0.002576,0.018550,-0.020472,0.001188,0.003661,0.008727,0.000685,-0.009755
Utilities_NoSeWa,0.010123,-0.001881,0.009994,-0.051552,-0.005560,-0.004930,-0.004163,0.008826,-1.000000,1.000000,0.006220,-0.005651,-0.002481,0.011612,-0.003856,-0.005031,-0.004561,-0.007619,-0.005031,-0.002576,-0.018550,0.020472,-0.001188,-0.003661,-0.008727,-0.000685,0.009755


In [56]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5   
)

X_train = preprocessing.scale(X_train)

X_test = preprocessing.scale(X_test)

In [57]:
params = {'kernel': 'linear'}
classifier = SVC(**params)
classifier.fit(X_train, y_train)
y_test_pred = classifier.predict(X_test)
mean_absolute_error(y_test_pred, y_test)

36709.53082191781

In [60]:
params = {'kernel': 'rbf', 'C':10.0, 'epsilon': 0.2}
regressor = SVR(**params)
regressor.fit(X_train, y_train)
y_test_pred = regressor.predict(X_test)
mean_absolute_error(y_test_pred, y_test)

54664.67172458467